### WTG Test

#### A simple test to probe the instantaneous WTG circulation response to a midlevel vortex vs. LW-CRF.

James Ruppert  
jruppert@ou.edu  
4/23/22

In [1]:
from netCDF4 import Dataset
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sys
from thermo_functions import theta_dry, theta_virtual, density_moist

#### Time selection

In [2]:
nt=6
t0 = 48
t1 = t0+nt

xmin=780

#### Directories

In [3]:
figdir = "/Users/jruppert/code/tc_figs/"
main = "/Users/jruppert/code/tc_output/"
storm = !ls $main

istorm=storm[0]
memb = !ls $main/$istorm/
imemb=memb[1]

datdir = main+istorm+'/'+imemb+'/ctl/'
print(datdir)

/Users/jruppert/code/tc_output/haiyan/memb_01/ctl/


#### Read variables

In [13]:
# Fill contour variable

# Vertical coordinate
filtmp = Dataset(datdir+'T.nc')
pres = filtmp.variables['pres'][:] # hPa
print("Vertical shape: ",np.shape(pres))
filtmp.close()
nz=np.shape(pres)[0]

# CRF
varfil_main = Dataset(datdir+'RTHRATLW.nc') # this opens the netcdf file
lwcrf = varfil_main.variables['RTHRATLW'][t0:t1,:,:,xmin:1400-1] * 3600.*24 # K/s --> K/d
varfil_main.close()
varcs = Dataset(datdir+'RTHRATLWC.nc') # this opens the netcdf file
cs = varcs.variables['RTHRATLWC'][t0:t1,:,:,xmin:1400-1] * 3600.*24 # K/s --> K/d
lwcrf -= cs
varcs.close()

# Temperature
varfil_main = Dataset(datdir+'T.nc')
tmpk = varfil_main.variables['T'][t0:t1,:,:,xmin:1400-1] # K
# theta = theta_dry(tmp,(pres[np.newaxis,:,np.newaxis,np.newaxis])*1e2) # K
varfil_main.close()

# Moisture
varfil_main = Dataset(datdir+'QVAPOR.nc')
qv = varfil_main.variables['QVAPOR'][t0:t1,:,:,xmin:1400-1] # kg/kg
varfil_main.close()

rho = np.mean( density_moist(tmpk,qv,pres[np.newaxis,:,np.newaxis,np.newaxis]*1e2), axis=(0,2,3))
tmp = np.mean(tmpk, axis=(0,2,3))

# Dry static energy
z=np.zeros(nz)
for iz in range(nz-1):
    z[iz+1] = z[iz] + (10000.) / (9.81 * (rho[iz-1] + rho[iz])*0.5)

dse = 1004.*tmp + 9.81*z

print(np.shape(tmpk))

Vertical shape:  (10,)
[302055.78125    310280.85290169 314283.6365211  318474.06692328
 322524.14817443 327094.05257926 330638.52395147 332280.06844293
 329987.64817617 331509.69280368]
(6, 10, 740, 619)


#### Analysis

$$ w\dfrac{\partial \theta}{\partial p} = Q_1$$

$$w_{wtg} = Q_1 \left( \dfrac{\partial \theta}{\partial p} \right) ^{-1}$$

In [14]:
# Heat tendency
qmn = np.sin(np.pi*np.arange(nz)/(nz-1))*10 * (1004./(3600.*24)) # K/d --> J/kg/s
print(qmn)

[0.         0.03590892 0.06830282 0.09401077 0.11051629 0.1162037
 0.11051629 0.09401077 0.06830282 0.03590892]


In [20]:
dsdp = np.gradient(dse,10000.)
wwtg = qmn / dsdp
print(wwtg)

[0.0 0.05873298049027086 0.16673023094173733 0.2281673154788071
 0.2564187320856458 0.28641440055687234 0.4262088374977186
 -2.888746967499535 -1.7732342462230886 0.23592553467037577]


In [25]:
tprm = np.sin(np.pi*np.arange(nz)/(nz-1))